In [ ]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('dataCapres/anies.csv')
df.head()

In [ ]:
df = df[['full_text', 'username', 'created_at']] # mengambil kolom full_text, username, dan created_at
df # menampilkan dataframe

## 1. Cleaning Data


In [ ]:
# kegunaannya untuk mengetahui jumlah baris dan kolom dari dataframe
df.shape 

In [ ]:
# menghapus data duplikat berdasarkan kolom full_text
df = df.drop_duplicates(subset=['full_text'])

In [ ]:
# mengecek apakah masih ada data duplikat
df.duplicated().sum()

In [ ]:
# menghapus baris yang memiliki nilai NaN atau null
df = df.dropna()

In [ ]:
# mengecek apakah masih ada nilai NaN atau null
df.isnull().sum()

In [ ]:
# mengecek apakah ada perubahan jumlah baris
df.shape

In [ ]:
# pembuatan function untuk melakukan pembersihan
def clean_twitter_texr(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#\w+', '', text) # menghapus hastag
    text = re.sub(r'RT[\s]+', '', text) # menghapus retweet
    text = re.sub(r'https?://\S+', '', text) # menghapus url/link

    text = re.sub(r'[^A-Za-z0-9]+', ' ', text) # menghapus simbol (non-alfanumerik)
    text = re.sub(r'\s+', ' ', text).strip() # menghapus spasi yang berlebihan

    return text

# menerapkan function untuk pembersihan teks
df['full_text'] = df['full_text'].apply(clean_twitter_texr)

In [ ]:
# mengubah teks menjadi huruf kecil
df['full_text'] = df['full_text'].str.lower() 

In [ ]:
# mengecek apakah ada perubahan
df

   ## 2. Preprocessing
   - Normalisasi
   - Stopword
   - Tokenize
   - Stemming



In [ ]:
# Normalisasi

# ini adalah proses mengganti kata yang salah dengan kata yang benar
# karena ini adalah bagian data anies, maka kita juga akan menghapus jika ada nama prabowo atau ganjar
# norm adalah dictionary yang berisi kata yang salah dan kata yang benar
norm = {" yg " : " yang ", 
        " ganjar " : " ", 
        " prabowo " : " ", 
        "pram " : " ", 
        " ngga " : " tidak ", 
        " nggak " : " tidak ", 
        " ga " : " tidak ", 
        " gak " : " tidak ", 
        " vibes " : " suasana ", 
        " text " : " teks ", 
        " mantab " : " keren ", 
        " end " : " selesai ", 
        " kelen " : " kalian ", 
        " jd " : " jadi ", 
        " tuk " : " untuk ", 
        " bangetdari " : " banget dari ", 
        " disampaikam " : " disampaikan ", 
        " kk " : " kakak "}

# membuat function normalisasi 
def normalisasi(str_text):
    # perulangan ini bertujuan untuk mengganti kata yang salah dengan kata yang benar diatas
    for i in norm:
        str_text = str_text.replace(i, norm[i])
    return str_text

# menerapkan function
df['full_text'] = df['full_text'].apply(lambda x: normalisasi(x))

# mengecek apakah ada perubahan
df

In [ ]:
# %pip install Sastrawi

In [ ]:
# Stopwords

import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

# ini akan membuang kata kata yang tidak memiliki bobot, contohnya kata di, yang, dengan, dan lain lain
# disini saya tambahkan kata "tidak", karena saya merasa ini juga tidak memiliki bobot
more_stop_words = ["tidak"]

stop_words = StopWordRemoverFactory().get_stop_words()
stop_words.extend(more_stop_words)

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
    str_text = stop_words_remover_new.remove(str_text)
    return str_text

df['full_text'] = df['full_text'].apply(lambda X: stopword(X))
df.head()

In [ ]:
# Tokenize

# kegunaan tokenize ini adalah untuk memecah teks menjadi kata-kata
tokenized = df['full_text'].apply(lambda x:x.split())
tokenized

In [ ]:
# Stemming

# # fungsinya untuk mengubah kata yang tadinya memiliki imbuhan menjadi kata dasar
# # contoh : menipu menjadi tipu

# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# def stemming(text_cleaning):
#     factory = StemmerFactory()
#     stemmer = factory.create_stemmer()
#     do = []
#     for w in text_cleaning:
#         dt = stemmer.stem(w)
#         do.append(dt)
#     d_clean = []
#     d_clean = ' '.join(do)
#     print(d_clean)
#     return d_clean

# tokenized = tokenized.apply(stemming)

# # karena proses ini lama, kita akan ngebackup data ke dalam csv, agar kedepannya tidak perlu melakukan proses ini lagi
# # menyimpan data ke dalam csv 

# tokenized.to_csv('dataPreprocessing/PreprocessingAnies.csv', index=False)


## 3. Translate

In [ ]:
# %pip install translate

In [ ]:
# memasukkan data hasil stemming ke dalam dataframe
data = pd.read_csv('dataPreprocessing/PreprocessingAnies.csv', encoding='latin-1')
data.head()

#kenapa atributnya cuma 1 yaitu full_text? karena untuk proses pengolahan labeling ataupun analisis datanya kita hanya memerlukan atribut komentar dari masyarakat yaitu full_text

In [ ]:
# from translate import Translator

# # fungsi ini berguna untuk mengubah teks dari bahasa indonesia menjadi bahasa inggris
# def convert_eng(tweet):
#     translator = Translator(to_lang="en", from_lang="id")
#     translation = translator.translate(tweet)
#     return translation

# # bagian mengapply fungsi di atas ke dalam data
# data['tweet_english'] = data['full_text'].apply(convert_eng)

# # menyimpan/backup data ke dalam csv, karena proses ini lumayan lama, jadi kedepannya agar tidak perlu running bagian ini lagi
# data.to_csv('dataTerjemahan/TerjemahanAnies.csv', index=False)

## 4. Labeling

In [ ]:
# memasukkan data hasil terjemahan ke dalam dataframe
data = pd.read_csv('dataTerjemahan/TerjemahanAnies.csv')
data

In [ ]:
# %pip install tweet-preprocessor
# %pip install textblob
# %pip install wordcloud
# %pip install nltk

In [ ]:
import preprocessor as p
from textblob import TextBlob
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
# nltk.download('all')


In [ ]:
data_tweet = list(data['tweet_english'])
polaritas = 0

status = [] # untuk melihat status sentimen
total_positif = total_negatif = total_netral = total = 0

# proses labeling
for i, tweet in enumerate(data_tweet):
    analysis = TextBlob(tweet)
    polaritas = analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif += 1
        status.append('Positif')
    elif analysis.sentiment.polarity == 0.0:
        total_netral += 1
        status.append('Netral')
    else:
        total_negatif += 1
        status.append('Negatif')

    total += 1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}')
print(f'\nTotal Data: {total}')

In [ ]:
data['klasifikasi'] = status
data

## 5. Visualisasi

In [ ]:
from wordcloud import WordCloud, STOPWORDS

# kita akan menampilkan visualisasi workcloud atau menampilkan beberapa data yang memiliki kuantitas yang tinggi
def plot_cloud(wordcloud):
    plt.figure(figsize=(10, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

all_words = ' '.join([tweet for tweet in data['full_text']])

wordcloud = WordCloud(
    width=3000, 
    height=2000, 
    random_state=3, 
    background_color='black', 
    colormap='Blues_r',
    collocations=False,
    stopwords=STOPWORDS
).generate(all_words)

plot_cloud(wordcloud)

In [ ]:
import seaborn as sns

sns.set_theme()

labels = ['Positif', 'Negatif', 'Netral']
counts = [total_positif, total_negatif, total_netral]

def show_bar_chart(labels, counts, title):
    fig, ax = plt.subplots(figsize=(8, 6))
    bars = ax.bar(labels, counts, color=['#2394f7', '#f72323', '#fac343'])

    for bar, count in zip(bars, counts):
        height = bar.get_height()
        ax.annotate(f'{count}', 
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

    ax.grid(axis='y', linestyle='--', alpha=0.7)
    ax.set_xlabel('Sentimen')
    ax.set_ylabel('Jumlah')
    ax.set_title(title)

plt.show()

show_bar_chart(labels, counts, "Distribusi Sentimen Anies Baswedan")

## 6. Klasifikasi Sentimen

In [ ]:
data

In [ ]:
dataset = data.drop('full_text', axis=1, inplace=False)
dataset = [tuple(x) for x in dataset.to_records(index=False)]

In [ ]:
import random

# inisialisasi sentiment
set_positif = []
set_negatif = []
set_neutral = []

# membagi data menjadi 3 bagian
# jika sentimennya positif, maka akan masuk ke set_positif, dan seterusnya
for n in dataset:
    if n[1] == 'Positif':
        set_positif.append(n)
    elif n[1] == 'Negatif':
        set_negatif.append(n)
    else:
        set_neutral.append(n)

# membagi data menjadi 2 bagian
set_positif = random.sample(set_positif, k=int(len(set_positif) / 2))
set_negatif = random.sample(set_negatif, k=int(len(set_negatif) / 2))
set_neutral = random.sample(set_neutral, k=int(len(set_neutral) / 2))

# menggabungkan set_positif, set_negatif, dan set_neutral, dan menyimpannya pada variabel train
train = set_positif + set_negatif + set_neutral

train_set = []

for n in train:
    train_set.append(n)

In [ ]:
# from nltk.classify import NaiveBayesClassifier
# from nltk.classify import accuracy

# # Format data menjadi format yang sesuai untuk classifier nltk
# train_data = [(dict([(word, True) for word in doc.split()]), category) for doc, category in train_set]

# # Melatih model
# classifier = NaiveBayesClassifier.train(train_data)

# # Menghitung akurasi
# test_data = [(dict([(word, True) for word in doc.split()]), category) for doc, category in dataset]
# print("Akurasi Test: ", accuracy(classifier, test_data))


In [ ]:
# import os
# import nltk

# # Ensure we have a clean, writable download path
# nltk_data_path = os.path.join(os.path.expanduser('~'), 'nltk_data')
# os.makedirs(nltk_data_path, exist_ok=True)

# # Explicitly set the data path
# nltk.data.path = [nltk_data_path]

# # Download multiple related NLTK resources
# nltk.download('punkt')
# nltk.download('punkt_tab')
# nltk.download('averaged_perceptron_tagger')

# # Verify downloads
# try:
#     # Force a download with a specific download directory
#     nltk.download('punkt', download_dir=nltk_data_path)
#     nltk.download('punkt_tab', download_dir=nltk_data_path)
    
#     # List available packages
#     print("Available NLTK data:")
#     for package in nltk.corpus.corpora.fileids():
#         print(package)
# except Exception as e:
#     print(f"Download error: {e}")

# # Try tokenization
# try:
#     from nltk.tokenize import sent_tokenize
    
#     text = "Hello world. This is a test sentence."
#     sentences = sent_tokenize(text)
#     print("\nTokenized sentences:", sentences)
# except Exception as e:
#     print(f"\nTokenization error: {e}")

In [ ]:
from textblob.classifiers import NaiveBayesClassifier

cl = NaiveBayesClassifier(train_set)
print("Akurasi Test: ", cl.accuracy(dataset))

In [ ]:
# Labeling
data_tweet = list(data['tweet_english'])
polaritas = []
status = []
total_positif = total_negatif = total_neutral = total = 0

for i, tweet in enumerate(data_tweet):
    analysis = TextBlob(tweet, classifier=cl)
    
    if analysis.classify() == 'Positif':
        total_positif += 1
    elif analysis.classify() == 'Neutral':
        total_neutral += 1
    else:
        total_negatif += 1

    status.append(analysis.classify())
    total += 1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNeutral = {total_neutral}\nNegatif = {total_negatif}')

In [ ]:
status = pd.DataFrame({"Klasifikasi Bayes": status})
data['klasifikasi_bayes'] = status

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set tema seaborn untuk tampilan yang lebih profesional
sns.set_theme()

labels = ['Positif', 'Negatif', 'Netral']
counts = [total_positif, total_negatif, total_neutral]

def show_bar_chart(labels, counts, title):
    fig, ax = plt.subplots(figsize=(8, 6))
    bars = ax.bar(labels, counts, color=['#2394f7', '#f72323', '#fac343'])

    # Menambahkan keterangan presentase
    for bar, count in zip(bars, counts):
        height = bar.get_height()
        ax.annotate(f'{count}', 
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

    # Menambahkan grid
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Menambahkan label sumbu dan judul
    ax.set_xlabel('Sentimen')
    ax.set_ylabel('Jumlah')
    ax.set_title(title)

plt.show()

show_bar_chart(labels, counts, "Distribusi Sentimen Ganjar Pranowo")


In [ ]:
data

In [ ]:
data_eval = [tuple(x) for x in data.to_records(index=False)]

for n in data_eval:
    if len(n) >= 4:
        if n[2] == n[3]:
            print(f'Text: {n[0]}\nClassifier: {n[2]}\nClassifier Bayes: {n[3]}\n')
    else:
        print("Tuple tidak memiliki cukup elemen.")
